In [5]:
%load_ext autoreload
%autoreload 2

In [1]:
from train_utils import *

In [21]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=8264, hidden_size=4096):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, 3, num_layers=1, batch_first=True)
        self.fc1 = nn.Linear(3, 1)
        self.fc2 = nn.Linear(1, input_size)

    def forward(self, x):
        batch_size = x.size(0)
        out, _ = self.lstm1(x)
        # out = out.contiguous().view(batch_size, -1)
        # out = self.fc1(out)
        out = self.fc1(out[:, -1, :])
        out = nn.functional.relu(out)
        out = torch.tanh(self.fc2(out))

        return out

In [27]:
data_dir = "/sdf/group/lcls/ds/scratch/s2e_scratch/Data/SFG_reIm/version1"
train_dataset = CustomSequence(data_dir, range(80), file_batch_size=1, model_batch_size=512)
val_dataset = CustomSequence(data_dir, range(80, 81), file_batch_size=1, model_batch_size=512)


In [28]:
model = LSTMModel()

In [29]:
train(model, train_dataset, num_epochs=10, val_dataset=val_dataset, use_gpu=True, data_parallel=True, out_dir=".", model_name="model", verbose=1, save_checkpoints=True, custom_loss=None)

Using 4 GPUs!
Epoch 1
haha
haha
haha
haha
haha
Epoch 1: Train Loss=0.217423045271635057, Val Loss=0.188546876859664925
Epoch 2
haha
haha
haha
haha
haha
Epoch 2: Train Loss=0.166685538816452039, Val Loss=0.145247697424888605


SystemExit: 0

/sdf/home/w/wmy/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [41]:
test_dataset = CustomSequence(data_dir, range(90, 92), file_batch_size=1, model_batch_size=512, test_mode=True)


In [42]:
predict(model, model_param_path="model_epoch_2.pth", test_dataset=test_dataset, use_gpu=True, data_parallel=False, output_dir=".", output_name="all_preds.npy", verbose=1)

Predicting batch 1/2
Predicting batch 2/2


In [44]:
np.load("all_preds.npy.npy").shape

(200, 8264)